## Загрузим нужные библиотеки

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score

%matplotlib inline

В этом примере мы попробуем предсказать итоговую оценку участинка только на основе **его действий**

## Пытаемся что-то выжать из информации о командах

In [2]:
df_user = pd.read_csv("./content/user.csv")

In [3]:
df_user.head()

,user_id,team_id,game_id
0,10189,1664,235
1,10018,1690,237
2,10580,1394,194
3,10600,1908,262
4,10112,1592,219


In [4]:
df_user = df_user.drop(['game_id'], axis=1)

In [5]:
df_user.head()

,user_id,team_id
0,10189,1664
1,10018,1690
2,10580,1394
3,10600,1908
4,10112,1592


### Подключаем файл с информацией с таблицами с баллами от комманд

In [8]:
df_team_point = pd.read_csv("./content/team_point.csv")

In [9]:
df_team_point.head()

,team_id,category_id,period,score,place
0,1948,2,0,16,2
1,1934,3,4,45,6
2,1894,4,8,24,4
3,1688,4,0,27,4
4,1592,4,2,21,5


In [11]:
### Категорию (category_id) и место (рейтингоывый номер) команды (place) отложим до лучших времен

In [12]:
df_team_point = df_team_point.drop(['category_id', 'place'], axis=1)

KeyError: "['category_id', 'place'] not found in axis"

In [14]:
df_team_point.head()

,team_id,period,score
0,1948,0,16
1,1934,4,45
2,1894,8,24
3,1688,0,27
4,1592,2,21


### На основании информации о том, какие участники в каких командах участвовали, попробуем узнать средний бал, который мог бы получить участник

In [25]:
#df_user_team_point = pd.concat([df_user, df_team_point], axis=1)
df_user_team_point = pd.merge(df_user, df_team_point, how="left", on=['team_id'])

In [27]:
df_user_team_point.head(40)

,user_id,team_id,period,score
0,10189,1664,1,105
1,10189,1664,3,33
2,10189,1664,2,17
3,10189,1664,1,4
4,10189,1664,1,21
5,10189,1664,0,8
6,10189,1664,3,151
7,10189,1664,3,1
8,10189,1664,2,170
9,10189,1664,4,86


In [32]:
#dfg_user_team = df_user_team_point['team_id', 'user_id']
dfg_user_team = df_user_team_point.groupby(['team_id', 'user_id']).size().reset_index(name='Freq')
dfg_user_team.head(20)

,team_id,user_id,Freq
0,1248,10273,30
1,1248,10646,30
2,1248,10762,30
3,1252,10547,30
4,1252,10780,30
5,1252,10833,30
6,1254,10276,30
7,1254,10420,30
8,1254,10426,30
9,1254,10892,30


### Выясним, сколько было участников в каждой команде

In [33]:
dfg_team_size = dfg_user_team.groupby(['team_id']).size().reset_index(name='Freq')

In [34]:
dfg_team_size.head()

,team_id,Freq
0,1248,3
1,1252,3
2,1254,4
3,1256,4
4,1258,5


### Теперь возвращаемся к первоначальному объединенному датасету и присоединяем к нему количество участников (с тем, чтобы потом высчитать средний бал по активности участника)

In [36]:
df_user_team_mean = pd.merge(df_user_team_point, dfg_team_size, how="left", on=['team_id'])

In [38]:
df_user_team_mean['team_user_mean'] = df_user_team_mean['score']/df_user_team_mean['Freq']

In [39]:
df_user_team_mean

,user_id,team_id,period,score,Freq,team_user_mean
0,10189,1664,1,105,5,21.00
1,10189,1664,3,33,5,6.60
2,10189,1664,2,17,5,3.40
3,10189,1664,1,4,5,0.80
4,10189,1664,1,21,5,4.20
...,...,...,...,...,...,...
29185,10471,1518,4,128,4,32.00
29186,10471,1518,2,42,4,10.50
29187,10471,1518,4,37,4,9.25
29188,10471,1518,1,225,4,56.25


### Удаляем team_id, score и Freq, т.е. избавляемся от информации о командах

In [40]:
df_user_team_mean = df_user_team_mean.drop(['team_id','score', 'Freq'], axis=1)

In [41]:
df_user_team_mean

,user_id,period,team_user_mean
0,10189,1,21.00
1,10189,3,6.60
2,10189,2,3.40
3,10189,1,0.80
4,10189,1,4.20
...,...,...,...
29185,10471,4,32.00
29186,10471,2,10.50
29187,10471,4,9.25
29188,10471,1,56.25


### А теперь вообще усредняем полученные средние баллы в команде для каждого участника по каждому периоду

In [61]:
df_user_team_mean_avg = df_user_team_mean.groupby(['user_id','period'])['team_user_mean'].mean().reset_index()

In [62]:
df_user_team_mean_avg

,user_id,period,team_user_mean
0,10001,0,10.533333
1,10001,1,19.400000
2,10001,2,16.400000
3,10001,3,21.866667
4,10001,4,12.066667
...,...,...,...
4860,10973,0,10.533333
4861,10973,1,12.733333
4862,10973,2,12.733333
4863,10973,3,-1.200000


## Возвращаемся к базовому решению

In [45]:
df_train = pd.read_csv("./content/train_dataset_train.csv")
df_user_decision = pd.read_csv("./content/user_decision.csv")

In [46]:
df_train.shape

(680, 5)

In [47]:
df_user.shape

(973, 2)

In [48]:
df_user.head(3)

,user_id,team_id
0,10189,1664
1,10018,1690
2,10580,1394


In [6]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60228 entries, 0 to 60227
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      60228 non-null  int64
 1   period       60228 non-null  int64
 2   decision_id  60228 non-null  int64
dtypes: int64(3)
memory usage: 1.4 MB


### Узнаем количество действий участника в каждом периоде

Узнаем частоту команд участника в каждом периоде

In [52]:
df_user_decision_train = df_user_decision[df_user_decision.user_id.map(lambda x: x in df_train.id.values)]
df_user_decision_train

,user_id,period,decision_id
0,10625,1,409
2,10775,4,420
3,10236,2,284
4,10130,3,72
5,10273,2,140
...,...,...,...
60220,10528,1,88
60222,10832,3,315
60224,10189,1,33
60226,10424,2,140


In [53]:
df_user_decision_count = df_user_decision_train.groupby(by=["user_id","period"]).count()

In [54]:
df_user_decision_count.head(5)

decision_id
user_id period             
10001   1                29
        2                23
        3                39
        4                15
10003   1                21

In [56]:
for id in df_train.id.values:
    for period in range(1,5):
        try :
            index = df_train[df_train.id == id].index[0]
            df_train.loc[index,"period_"+str(period)] = df_user_decision_count.loc[pd.IndexSlice[id, period:period], :].values[0][0]
        except IndexError:
            continue

In [58]:
df_train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4
0,10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0
1,10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0
2,10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0
3,10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0
4,10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0
5,10908,4.0,4.0,5.0,5.0,NaN,4.0,10.0,7.0
6,10378,4.0,5.0,5.0,4.0,3.0,1.0,NaN,NaN
7,10220,5.0,4.0,5.0,5.0,1.0,NaN,2.0,1.0
8,10612,5.0,4.0,4.0,5.0,26.0,22.0,12.0,15.0
9,10776,4.0,4.0,5.0,5.0,16.0,13.0,37.0,36.0


### Добавим колонки со средним значением по команде по периоду

In [63]:
df_train['team_period_1'] = 0
df_train['team_period_2'] = 0
df_train['team_period_3'] = 0
df_train['team_period_4'] = 0

In [69]:
def get_periods_from_user_team_mean(id):
    res = [0,0,0,0]
    df = df_user_team_mean_avg[df_user_team_mean_avg['user_id']==id]
    for i in range(len(df)):
        if 0 <= df.iloc[i, 1] <= 3:
            res[df.iloc[i, 1]] = df.iloc[i, 2]
    return res

In [72]:
for i in range(len(df_train)):
    res = get_periods_from_user_team_mean(df_train.iloc[i,df_train.columns.get_loc('id')])
    df_train.iloc[i,df_train.columns.get_loc('team_period_1')] = res[0]
    df_train.iloc[i,df_train.columns.get_loc('team_period_2')] = res[1]
    df_train.iloc[i,df_train.columns.get_loc('team_period_3')] = res[2]
    df_train.iloc[i,df_train.columns.get_loc('team_period_4')] = res[3]

In [73]:
df_train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,team_period_1,team_period_2,team_period_3,team_period_4
0,10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0,13.166667,11.833333,6.916667,5.000000
1,10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0,10.533333,11.266667,15.600000,13.266667
2,10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0,10.533333,6.133333,19.933333,11.333333
3,10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0,10.533333,5.466667,5.333333,10.000000
4,10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0,13.166667,23.250000,24.000000,6.250000
5,10908,4.0,4.0,5.0,5.0,NaN,4.0,10.0,7.0,10.533333,10.666667,12.333333,7.666667
6,10378,4.0,5.0,5.0,4.0,3.0,1.0,NaN,NaN,13.166667,17.000000,22.166667,15.000000
7,10220,5.0,4.0,5.0,5.0,1.0,NaN,2.0,1.0,10.533333,7.600000,10.800000,11.333333
8,10612,5.0,4.0,4.0,5.0,26.0,22.0,12.0,15.0,13.166667,17.500000,19.916667,18.000000
9,10776,4.0,4.0,5.0,5.0,16.0,13.0,37.0,36.0,26.333333,18.000000,21.500000,10.666667


In [75]:
df_train = df_train.fillna(0)

In [76]:
df_train = df_train.set_index('id')

In [77]:
df_train

,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,team_period_1,team_period_2,team_period_3,team_period_4
id,,,,,,,,,,,,
10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0,13.166667,11.833333,6.916667,5.000000
10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0,10.533333,11.266667,15.600000,13.266667
10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0,10.533333,6.133333,19.933333,11.333333
10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0,10.533333,5.466667,5.333333,10.000000
10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0,13.166667,23.250000,24.000000,6.250000
...,...,...,...,...,...,...,...,...,...,...,...,...
10563,5.0,4.0,5.0,4.0,10.0,2.0,14.0,4.0,10.533333,7.533333,5.800000,3.400000
10386,5.0,4.0,5.0,5.0,1.0,14.0,24.0,1.0,10.533333,12.133333,9.400000,13.200000
10675,5.0,4.0,5.0,4.0,4.0,17.0,23.0,12.0,10.533333,11.266667,8.133333,7.666667


## Выделим выборки

In [78]:
X = df_train.drop(["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"], axis = 1)
y = df_train[["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]]

In [79]:
X.shape

(680, 8)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели

In [83]:
clf = RandomForestClassifier(random_state=42)

In [84]:
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Оценка точности

In [85]:
pred = clf.predict(X_test)

In [86]:
y_test.head(3)

,Analytical thinking,Systemic thinking,Adaptability,Focus
id,,,,
10641,5.0,5.0,6.0,5.0
10026,5.0,4.0,5.0,5.0
10548,4.0,4.0,5.0,4.0


In [88]:
result = 0 

result += recall_score(y_test["Analytical thinking"], pred[:,0], average='macro', zero_division=True)
result += recall_score(y_test["Systemic thinking"], pred[:,1], average='macro', zero_division=True)
result += recall_score(y_test["Adaptability"], pred[:,2], average='macro', zero_division=True)
result += recall_score(y_test["Focus"], pred[:,3], average='macro', zero_division=True)

print("Recall score",result/4)

Recall score 0.2563485184360177


## Снова считаем, но еще точнее

In [89]:
clf.fit(X, y)

RandomForestClassifier(random_state=42)

## А теперь готовим финальный результат

In [127]:
df_solution = pd.read_csv("./content/sample_solution.csv")

In [128]:
df_solution

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10199,0,0,0,0
1,10539,0,0,0,0
2,10174,0,0,0,0
3,10465,0,0,0,0
4,10066,0,0,0,0
...,...,...,...,...,...
288,10433,0,0,0,0
289,10893,0,0,0,0
290,10909,0,0,0,0
291,10889,0,0,0,0


In [129]:
df_user_decision_solution = df_user_decision[df_user_decision.user_id.map(lambda x: x in df_solution.id.values)]

In [130]:
df_user_decision_solution

,user_id,period,decision_id
1,10318,3,203
6,10617,3,329
9,10316,3,26
13,10850,4,255
17,10754,3,407
...,...,...,...
60214,10630,4,26
60216,10848,2,409
60221,10054,1,343
60223,10312,3,329


In [131]:
df_user_decision_solution_count = df_user_decision_solution.groupby(by=["user_id","period"]).count()

In [132]:
for id in df_solution.id.values:
    for period in range(1,5):
        try :
            index = df_solution[df_solution.id == id].index[0]
            df_solution.loc[index,"period_"+str(period)] = df_user_decision_solution_count.loc[pd.IndexSlice[id, period:period], :].values[0][0]
        except IndexError:
            continue

In [133]:
df_solution

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4
0,10199,0,0,0,0,2.0,2.0,3.0,2.0
1,10539,0,0,0,0,1.0,9.0,15.0,17.0
2,10174,0,0,0,0,NaN,28.0,31.0,20.0
3,10465,0,0,0,0,15.0,18.0,32.0,18.0
4,10066,0,0,0,0,8.0,23.0,16.0,6.0
...,...,...,...,...,...,...,...,...,...
288,10433,0,0,0,0,18.0,7.0,1.0,NaN
289,10893,0,0,0,0,21.0,52.0,34.0,20.0
290,10909,0,0,0,0,NaN,NaN,NaN,1.0
291,10889,0,0,0,0,6.0,2.0,5.0,1.0


In [134]:
df_solution['team_period_1'] = 0
df_solution['team_period_2'] = 0
df_solution['team_period_3'] = 0
df_solution['team_period_4'] = 0

In [135]:
for i in range(len(df_solution)):
    res = get_periods_from_user_team_mean(df_solution.iloc[i,df_solution.columns.get_loc('id')])
    df_solution.iloc[i,df_solution.columns.get_loc('team_period_1')] = res[0]
    df_solution.iloc[i,df_solution.columns.get_loc('team_period_2')] = res[1]
    df_solution.iloc[i,df_solution.columns.get_loc('team_period_3')] = res[2]
    df_solution.iloc[i,df_solution.columns.get_loc('team_period_4')] = res[3]

In [136]:
df_solution

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,team_period_1,team_period_2,team_period_3,team_period_4
0,10199,0,0,0,0,2.0,2.0,3.0,2.0,13.166667,16.833333,13.750000,7.583333
1,10539,0,0,0,0,1.0,9.0,15.0,17.0,52.666667,35.666667,60.333333,13.666667
2,10174,0,0,0,0,NaN,28.0,31.0,20.0,13.166667,23.083333,26.500000,17.083333
3,10465,0,0,0,0,15.0,18.0,32.0,18.0,13.166667,13.416667,12.000000,6.666667
4,10066,0,0,0,0,8.0,23.0,16.0,6.0,10.533333,8.866667,0.800000,1.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,10433,0,0,0,0,18.0,7.0,1.0,NaN,10.533333,17.066667,9.733333,21.800000
289,10893,0,0,0,0,21.0,52.0,34.0,20.0,10.533333,7.933333,11.666667,21.000000
290,10909,0,0,0,0,NaN,NaN,NaN,1.0,8.777778,9.944444,9.111111,10.444444
291,10889,0,0,0,0,6.0,2.0,5.0,1.0,10.533333,7.933333,17.600000,20.333333


In [137]:
df_solution = df_solution.fillna(0)

In [138]:
df_solution = df_solution.set_index('id')

In [149]:
X_solution = df_solution.drop(["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"], axis = 1)
y_solution = df_solution[["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]]

In [150]:
y_pred = clf.predict(X_solution)

In [151]:
y_pred

array([[4., 4., 5., 4.],
       [4., 4., 5., 4.],
       [4., 4., 5., 4.],
       ...,
       [3., 4., 3., 3.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]])

In [152]:
y_solution.loc[:,["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]] = y_pred

C:\Users\Сергей\AppData\Local\Temp\ipykernel_6536\2406869456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_solution.loc[:,["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]] = y_pred


In [153]:
y_solution

,Analytical thinking,Systemic thinking,Adaptability,Focus
id,,,,
10199,4,4,5,4
10539,4,4,5,4
10174,4,4,5,4
10465,4,4,5,5
10066,4,4,5,4
...,...,...,...,...
10433,4,4,4,4
10893,4,4,5,5
10909,3,4,3,3


In [154]:
y_solution = y_solution.astype(int)

In [155]:
y_solution

,Analytical thinking,Systemic thinking,Adaptability,Focus
id,,,,
10199,4,4,5,4
10539,4,4,5,4
10174,4,4,5,4
10465,4,4,5,5
10066,4,4,5,4
...,...,...,...,...
10433,4,4,4,4
10893,4,4,5,5
10909,3,4,3,3


In [156]:
y_solution = y_solution.reset_index()

In [157]:
df_submission = y_solution

In [158]:
df_submission.to_csv("./content/RSF_solution.csv", index=False)